# Machine Learning

- Primero importamos la data en un dataframe para poder trabajar con ellos en los modelos de Machine Learning.
- Acontinuación genero las mascaras para enfocarme en la región de la imagen en la cual estoy interesado.
- Importa la librería rgbhistogram para utilizar la representación de intensidad de pixeles en la región de interes
- Separo la data en 0.75 entrenamiento y 0.25 para test
- Se procede a evaluar la data en diferentes modelos de Machine Learning y determinar cual da mejor resultado

In [1]:
#Importando las librerías
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
import rgbhistogram
from PIL import Image
from rembg import remove
from sklearn import metrics
import numpy as np
import pandas as pd
import os
import cv2

In [2]:
#Cargando la data
train_data = pd.read_csv("train.csv", thousands=',')

In [3]:
#Imprimiendo los primeros cinco registros de train
train_data.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [4]:
#Imprimiendo el número de registros en train
len(train_data)

1821

In [5]:
#Generación de máscaras
n = len(train_data)
for i in range(0,n):
    img = Image.open(os.path.join("images",train_data["image_id"][i]+".jpg"))
    output = remove(img)
    (T, thresh) = cv2.threshold(np.array(output), 10, 255, cv2.THRESH_BINARY)
    gray_thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join("mask","mask_"+train_data["image_id"][i]+".png"),gray_thresh)

In [6]:
#Guardando en listas imagenes y mascaras 
imagePaths = []
maskPaths = []
n = len(train_data)
for i in range(0, n):
    imagePaths.append("train\\" + train_data["image_id"][i] + ".jpg")
    maskPaths.append("mask\\mask_" + train_data["image_id"][i] + ".png")

In [7]:
#Usando histograma RGB para caracterizar los colores
desc = rgbhistogram.RGBHistogram([8, 8, 8])
data_desc = []
target = []
i = 0
for (imagePath, maskPath) in zip(imagePaths, maskPaths):
    image = cv2.imread(imagePath)
    mask = cv2.imread(maskPath)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    features = desc.describe(image, mask)
    data_desc.append(features)
    target.append(list(train_data.drop(columns="image_id").iloc[i]))
    i = i +1

In [8]:
#Creando lista de target con nombres únicos 
targetNames = np.unique(list(train_data.drop(columns="image_id")))

In [9]:
targetNames

array(['healthy', 'multiple_diseases', 'rust', 'scab'], dtype='<U17')

In [10]:
#Separación de datos: 0.75 para entrenamiento y 0.25 para test
(trainData, testData, trainTarget, testTarget) = train_test_split(data_desc, target, test_size = 0.25, random_state = 42)

CLASIFICADOR RANDOM FOREST 

In [11]:
model_RF = RandomForestClassifier(n_estimators = 25, random_state = 84)
    
model_RF.fit(trainData, trainTarget)
    
print(classification_report(testTarget, model_RF.predict(testData), target_names = targetNames))

                   precision    recall  f1-score   support

          healthy       0.78      0.76      0.77       126
multiple_diseases       1.00      0.05      0.09        22
             rust       0.96      0.87      0.91       155
             scab       0.84      0.73      0.78       153

        micro avg       0.86      0.75      0.80       456
        macro avg       0.89      0.60      0.64       456
     weighted avg       0.87      0.75      0.79       456
      samples avg       0.75      0.75      0.75       456



C:\Users\Franz\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
print("Error medio cuadrado")
print(metrics.mean_squared_error(testTarget,model_RF.predict(testData)))

Error medio cuadrado
0.09155701754385964


CLASIFICADOR DE K VECINOS MÁS CERCANOS

In [13]:
#Usando el clasificador de K vecinos más cercano
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(trainData, trainTarget)
Y_pred_kk = knn.predict(testData)
print("Presición de vecinos más cercanos")
print(knn.score(testData,testTarget))
print("Error medio cuadrado")
print(metrics.mean_squared_error(testTarget,Y_pred_kk))

Presición de vecinos más cercanos
0.45394736842105265
Error medio cuadrado
0.23081140350877194


C:\Users\Franz\AppData\Roaming\Python\Python39\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Franz\AppData\Roaming\Python\Python39\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will

CLASIFICADOR DE ÁRBOLES DE DESICIÓN

In [14]:
#Usando el clasificador árboles de desición
tree=DecisionTreeClassifier(max_depth=5)
tree.fit(trainData, trainTarget)
Y_pred_three = tree.predict(testData)
print("Presición de SGD")
print(tree.score(testData,testTarget))
print("Error medio cuadrado")
print(metrics.mean_squared_error(testTarget,Y_pred_three))

Presición de SGD
0.7214912280701754
Error medio cuadrado
0.1304824561403509


# Conclusión

El clasificador Random Forest nos da una mejor presición en la predicción, micro-average de 0.86.